<a href="https://colab.research.google.com/github/Gis-kost/geocod/blob/main/geod_Nominatim%2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 39.4 MB/s eta 0:00:00


In [ ]:
# 1. ЗАГРУЗКА ФАЙЛА
# ------------------
from google.colab import files
import pandas as pd
import requests
import time  # Добавлено для геокодирования

uploaded = files.upload()
input_filename = list(uploaded.keys())[0]

# Чтение файла с сохранением первого столбца как 'id'
df = pd.read_excel(input_filename, header=None, names=['id', 'original'], skiprows=1)
df = df.dropna(subset=['original']).reset_index(drop=True)
display("Превью загруженных данных:", df.head())


Saving addresses.xlsx to addresses (1).xlsx


'Превью загруженных данных:'

,id,original
0,1,"Тюмень, ул. Ленина, 10"
1,2,"Тюмень, ул. Республики, 142"
2,3,"Тюмень, ул. Первомайская, 20"
3,4,"Тюмень, ул. Холодильная, 57"
4,5,"Тюмень, ул. Мельникайте, 123а"


In [ ]:
# 2. ОПТИМИЗАЦИЯ АДРЕСОВ
# ----------------------
import re
from Levenshtein import ratio as similarity_ratio

def normalize_address(address):
    original = str(address).strip()
    normalized = re.sub(r'\s+', ' ', original)

    replacements = {
        r'\bул\.?(?=\s|$)': 'улица',
        r'\bпер\.?(?=\s|$)': 'переулок',
        r'\bпр-?т\.?(?=\s|$)': 'проспект',
        r'\bд\.?(?=\s|$)': 'дом',
        r'\bкорп\.?(?=\s|$)': 'корпус',
        r'\bг\.?(?=\s|$)': 'город',
        r'[^а-яё0-9\s,/-]': ''
    }

    for pattern, repl in replacements.items():
        normalized = re.sub(pattern, repl, normalized, flags=re.IGNORECASE)

    normalized = re.sub(r'(\d+)[/\\]\d+', r'\1', normalized)
    normalized = re.sub(r'\s+,', ',', normalized)
    normalized = normalized.title()
    similarity = round(similarity_ratio(original.lower(), normalized.lower()) * 100, 1)

    return pd.Series([normalized, similarity])

df[['normalized', 'similarity']] = df['original'].apply(normalize_address)
df.head()

,id,original,normalized,similarity
0,1,"Тюмень, ул. Ленина, 10","Тюмень, Улица Ленина, 10",91.3
1,2,"Тюмень, ул. Республики, 142","Тюмень, Улица Республики, 142",92.9
2,3,"Тюмень, ул. Первомайская, 20","Тюмень, Улица Первомайская, 20",93.1
3,4,"Тюмень, ул. Холодильная, 57","Тюмень, Улица Холодильная, 57",92.9
4,5,"Тюмень, ул. Мельникайте, 123а","Тюмень, Улица Мельникайте, 123А",93.3


In [ ]:
# 3. ГЕОКОДИРОВАНИЕ
# ---------------------------------------
def geocode(address):
    try:
        response = requests.get(
            "https://nominatim.openstreetmap.org/search",
            params={
                'q': address,
                'format': 'json',
                'countrycodes': 'ru',
                'addressdetails': 1
            },
            headers={'User-Agent': 'MyApp/1.0'}
        )
        if response.status_code == 200:
            data = response.json()
            if data:
                lat = round(float(data[0]['lat']), 6)
                lon = round(float(data[0]['lon']), 6)
                return lat, lon
        time.sleep(1)
    except Exception as e:
        print(f"Ошибка геокодирования: {e}")
    return None, None

df['lat'], df['lon'] = zip(*df['normalized'].apply(geocode))
display("Результаты геокодирования:", df)

'Результаты геокодирования:'

,id,original,normalized,similarity,lat,lon
0,1,"Тюмень, ул. Ленина, 10","Тюмень, Улица Ленина, 10",91.3,57.159788,65.523075
1,2,"Тюмень, ул. Республики, 142","Тюмень, Улица Республики, 142",92.9,57.137731,65.566580
2,3,"Тюмень, ул. Первомайская, 20","Тюмень, Улица Первомайская, 20",93.1,57.153447,65.532688
3,4,"Тюмень, ул. Холодильная, 57","Тюмень, Улица Холодильная, 57",92.9,57.146090,65.571739
4,5,"Тюмень, ул. Мельникайте, 123а","Тюмень, Улица Мельникайте, 123А",93.3,57.127649,65.557810
5,6,"Тюмень, ул. Минская, 45/1","Тюмень, Улица Минская, 45",88.0,57.146475,65.574791
6,7,"Тюмень, площадь Борцов Революции, 1","Тюмень, Площадь Борцов Революции, 1",100.0,57.159856,65.528679
7,8,"Тюмень, ул. 50 лет Октября, 63","Тюмень, Улица 50 Лет Октября, 63",93.5,57.137166,65.599077
8,9,"Тюмень, ул. Профсоюзная, 52/1","Тюмень, Улица Профсоюзная, 52",89.7,57.152558,65.555803
9,10,"Тюмень, ул. Широтная, 118/1","Тюмень, Улица Широтная, 118",88.9,57.105301,65.584454


In [ ]:
# 4. ИНТЕРАКТИВНАЯ КАРТА
# ---------------------
import folium
from IPython.display import display, HTML

def create_map():
    valid_points = df.dropna(subset=['lat', 'lon'])
    if not valid_points.empty:
        center = [valid_points['lat'].mean(), valid_points['lon'].mean()]
        m = folium.Map(location=center, zoom_start=12)
        for _, row in valid_points.iterrows():
            folium.Marker(
                location=[row['lat'], row['lon']],
                popup=f"<b>ID:</b> {row['id']}<br><b>Адрес:</b> {row['original']}",
                icon=folium.Icon(color='blue')
            ).add_to(m)
        display(m)
    else:
        display("Нет данных для карты")

create_map()

In [ ]:
# 5. СОХРАНЕНИЕ РЕЗУЛЬТАТОВ
# -------------------------
output_filename = 'geocoded_results.xlsx'
# Сохраняем столбец 'id' первым
df.to_excel(output_filename, index=False, columns=['id', 'original', 'normalized', 'similarity', 'lat', 'lon'])
files.download(output_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# добавить скачку веб карты
# добавить пересечение со зданиями, выводить таблицу